In [17]:
using Revise

In [47]:
using Plots; gr()

Plots.GRBackend()

In [238]:
using Flux
using CoordinateTransformations
import FluxExtensions
reload("FluxExtensions")

struct TangentPropagator{F <: Function, C <: Chain} # /home/rdeits/locomotion/explorations/learning-mpc/modules/FluxExtensions.jl, line 17:
    f::F # /home/rdeits/locomotion/explorations/learning-mpc/modules/FluxExtensions.jl, line 18:
    chain::C
end
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/FluxExtensions.ji for module FluxExtensions.


In [265]:
m = Chain(
    AffineMap(fill(2.0, 1, 1), [0.0]),
    Dense(1, 1, elu),
    Dense(1, 1, elu)
    )

Chain(AffineMap([2.0], [0.0]), Dense(1, 1, NNlib.elu), Dense(1, 1, NNlib.elu))

In [266]:
mp = FluxExtensions.TangentPropagator(m)

FluxExtensions.TangentPropagator{FluxExtensions.##3#4{Base.##55#56{Base.##55#56{Base.##55#56{Base.#identity,FluxExtensions.##7#8{Flux.Dense{NNlib.#elu,TrackedArray{…,Array{Float64,2}},TrackedArray{…,Array{Float64,1}}}}},FluxExtensions.##7#8{Flux.Dense{NNlib.#elu,TrackedArray{…,Array{Float64,2}},TrackedArray{…,Array{Float64,1}}}}},FluxExtensions.##5#6{CoordinateTransformations.AffineMap{Array{Float64,2},Array{Float64,1}}}}},Flux.Chain}(FluxExtensions.#3, Chain(AffineMap([2.0], [0.0]), Dense(1, 1, NNlib.elu), Dense(1, 1, NNlib.elu)))

In [267]:
mp([-1.0])

(param([-0.771514]), param([0.0996623]))

In [268]:
m([-1.0])

Tracked 1-element Array{Float64,1}:
 -0.771514

In [269]:
(m([-1 + 1e-4]) - m([-1.0])) / 1e-4

Tracked 1-element Array{Float64,1}:
 0.0996749

In [270]:
lf = (x, y, J) -> begin
    ŷ, Ĵ = mp(x)
    Flux.mse(ŷ, y) + Flux.mse(Ĵ, J)
end

(::#15) (generic function with 1 method)

In [271]:
train_data = [
    ([1.0], [1.2], [1.5])
    ]

1-element Array{Tuple{Array{Float64,1},Array{Float64,1},Array{Float64,1}},1}:
 ([1.0], [1.2], [1.5])

In [272]:
opt = Flux.Optimise.Momentum(params(mp))

(::#71) (generic function with 1 method)

In [275]:
for i in 1:1000
    Flux.train!(lf, train_data, opt)
end

In [276]:
mp([1.0])

(param([1.2]), param([1.5]))

In [182]:
@edit elu(1.0)

In [114]:
train_data =
    [
        ([0.0], [0.0]),
        ([1.0], [2.0]),
    ]

2-element Array{Tuple{Array{Float64,1},Array{Float64,1}},1}:
 ([0.0], [0.0])
 ([1.0], [2.0])

In [115]:
opt = Flux.Optimise.Momentum(params(m))

(::#71) (generic function with 1 method)

In [116]:
for i in 1:1000
    Flux.train!(loss, train_data, opt)
end

In [117]:
params(m)

2-element Array{Any,1}:
 param([1.0])         
 param([-2.59629e-17])

In [118]:
m([0.0])

Tracked 1-element Array{Float64,1}:
 0.0

In [119]:
m([1.0])

Tracked 1-element Array{Float64,1}:
 2.0

In [120]:
p = params(m)[1]

Tracked 1×1 Array{Float64,2}:
 1.0

In [121]:
f = FluxExtensions.plain(m)

(::#55) (generic function with 1 method)

In [122]:
f([1.0])

1-element Array{Float64,1}:
 2.0

In [123]:
ForwardDiff.jacobian(f, [1.0])

1×1 Array{Float64,2}:
 2.0

LoadError: [91minvalid redefinition of constant TangentPropagator[39m